In [72]:
import pandas as pd
import numpy as np
import pybaseball

In [73]:
from pybaseball import pitching_stats
pitcher_data = pitching_stats(2022, qual=5)
pitcher_data = pitcher_data[['IDfg', 'Name', 'Age', 'G', 'IP', 'WAR']]

In [74]:
from pybaseball import batting_stats
batter_data = batting_stats(2022, qual=20)
batter_data = batter_data[['IDfg', 'Name', 'Age', 'G', 'AB', 'WAR']]

In [75]:
salaries = pd.read_csv("../Data/MLB-Salaries 2000-23 - 2023.csv", header= 1).drop("MLS", axis = 1)
salaries = salaries[salaries['2023'].notna()]

In [76]:
appearances = pd.read_csv("../Data/PositionAppearances2022.csv")
appearances = appearances.drop(['yearID','teamID', 'lgID', 'G_batting', 'G_defense', "G_ph", "G_pr"], axis = 1).groupby('playerID', as_index=False).sum() 

In [77]:
from pybaseball import playerid_reverse_lookup
lookuptable = playerid_reverse_lookup(appearances['playerID'], key_type='bbref')

batter_data = batter_data.merge(lookuptable, left_on= "IDfg", right_on= "key_fangraphs")
batter_data.drop(['key_retro', 'key_mlbam', 'mlb_played_last'], axis = 1, inplace= True)
pitcher_data = pitcher_data.merge(lookuptable, left_on= "IDfg", right_on= "key_fangraphs")
pitcher_data.drop(['key_retro', 'key_mlbam', 'mlb_played_last'], axis = 1, inplace= True)


In [78]:
# This takes a while
#from pybaseball import playerid_lookup
#df = pd.DataFrame()
#for id, row in salaries.iterrows():
#    last = f"{row['Last']}"
#    first = f"{row['First'].strip()}" 
#    a = playerid_lookup(last, first, fuzzy= True).iloc[[0]]
#    df = pd.concat([df, a], ignore_index=True)
    

In [79]:
#df.to_csv('../Data/SalaryLookup.csv')
df = pd.read_csv('../Data/SalaryLookup.csv')

In [80]:
salaries["key_fangraphs"] = df["key_fangraphs"]

In [81]:
batter_data = batter_data.merge(salaries[['2023', 'key_fangraphs']],  on= "key_fangraphs")
pitcher_data = pitcher_data.merge(salaries[['2023', 'key_fangraphs']],  on= "key_fangraphs")

In [82]:
import statsapi
teams = statsapi.lookup_team("", activeStatus="Y") 

In [83]:
team_ids = [team['id'] for team in teams]

In [84]:
# Pulling rosters from Opening Day 2023
rosters_df = pd.DataFrame()
for id in team_ids: 
    roster_string = statsapi.roster(id,'40Man',date=statsapi.get('season',{'seasonId':2023,'sportId':1})['seasons'][0]['regularSeasonStartDate'])
    rows = [line.split(maxsplit=2) for line in roster_string.strip().split('\n')]
    df = pd.DataFrame(rows, columns=['Number', 'Position', 'Player'])
    df[['First Name', 'Last Name']] = df['Player'].str.split(n=1, expand=True)
    df['teamid'] = id
    rosters_df = pd.concat([rosters_df, df], ignore_index=True)


In [85]:
rosters_df
rosters_df = rosters_df.merge(pd.DataFrame(teams)[['id', 'shortName']], left_on= "teamid", right_on= "id").drop(["id"], axis = 1)

In [86]:
# This takes a while - mapping players with teams back to ids 
#from pybaseball import playerid_lookup
#mapped_rosters = pd.DataFrame()
#for id, row in rosters_df.iterrows():
#    last = f"{row['Last Name']}"
#    first = f"{row['First Name'].strip()}" 
#    a = playerid_lookup(last, first, fuzzy= True).iloc[[0]] 
#    a['Last Name'] = row['Last Name']
#    a['First Name'] = row['First Name']
#    a['shortName'] = row['shortName']
#    a['Number'] = row['Number']            
#    mapped_rosters = pd.concat([mapped_rosters, a], ignore_index=True)

In [87]:
#mapped_rosters.to_csv('../Data/RosterLookup.csv')
mapped_rosters = pd.read_csv('../Data/RosterLookup.csv')

In [88]:
mapped_rosters = mapped_rosters[['key_fangraphs', 'shortName' ]]

In [89]:
batter_data = batter_data.merge(mapped_rosters ,  on= "key_fangraphs")
pitcher_data = pitcher_data.merge(mapped_rosters , on= "key_fangraphs")

In [90]:
import os 

folder_path = '../Data/2023 OD Contracts/'
futurecontracts_df = pd.DataFrame()

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'): 

        df = pd.read_excel(os.path.join(folder_path, filename), skiprows=7)
        df = df.dropna(subset=['Unnamed: 1'])

        df_subset = df[['Unnamed: 0', 'Unnamed: 1', 2024, 2025, 2026, '2027']].rename(columns={'Unnamed: 0': 'Name', 'Unnamed: 1': 'Position'})

        df_subset.loc[df_subset['Position'].str.contains('rhp-s|lhp-s', na=False, case=False), 'Position'] = 'Starter'
        df_subset.loc[df_subset['Position'].isin(['rhp', 'lhp', 'rhp-c', 'lhp-c']), 'Position'] = 'Reliever'
        df_subset.loc[~df_subset['Position'].isin(['Starter', 'Reliever']), 'Position'] = 'Batter'

        df_subset['S2'] = df[[2024, 2025, 2026, '2027']].eq('A4').any(axis=1)

        futurecontracts_df = pd.concat([futurecontracts_df, df_subset], ignore_index=True) 


In [91]:
futurecontracts_df[['Last', 'First']] = futurecontracts_df['Name'].str.split(', ', expand=True)

In [92]:
# This takes a while
from pybaseball import playerid_lookup
future_mapped = pd.DataFrame()
for id, row in futurecontracts_df.iterrows():
    last = f"{row['Last']}"
    first = f"{row['First']}" 
    a = playerid_lookup(last, first, fuzzy= True).iloc[[0]]
    a['2024'] = row[2024]
    a['S2'] = row['S2']
    a['SimplePosition'] = row['Position']     
    future_mapped = pd.concat([future_mapped, a], ignore_index=True)
 

No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar

In [93]:
future_mapped = future_mapped[['key_fangraphs', '2024', 'SimplePosition' , 'S2']]

In [94]:
batter_data = batter_data.merge(future_mapped ,  on= "key_fangraphs")
pitcher_data = pitcher_data.merge(future_mapped , on= "key_fangraphs")

In [95]:
batter_data = batter_data.merge(appearances ,  left_on= "key_bbref", right_on = "playerID") 

In [96]:
batter_data

,IDfg,Name,Age,G,AB,WAR,name_last,name_first,key_bbref,key_fangraphs,mlb_played_first,2023,shortName,2024,SimplePosition,S2,playerID,G_all,GS,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh
0,11493,Manny Machado,29,150,578,7.5,machado,manny,machama01,11493,2012.0,"$21,090,909",San Diego,$17.091,Batter,False,machama01,150,148,0,0,0,0,134,0,0,0,0,0,15
1,9777,Nolan Arenado,31,148,557,7.2,arenado,nolan,arenano01,9777,2013.0,"$32,822,071",St. Louis,$35.000,Batter,False,arenano01,148,147,0,0,0,0,131,0,0,0,0,0,17
2,5361,Freddie Freeman,32,159,612,7.1,freeman,freddie,freemfr01,5361,2010.0,"$25,290,674",LA Dodgers,$25.291,Batter,False,freemfr01,159,159,0,0,159,0,0,0,0,0,0,0,0
3,9218,Paul Goldschmidt,34,151,561,7.0,goldschmidt,paul,goldspa01,9218,2011.0,"$25,333,333",St. Louis,$25.333,Batter,False,goldspa01,151,150,0,0,128,0,0,0,0,0,0,0,23
4,19556,Yordan Alvarez,25,135,470,6.7,álvarez,yordan,alvaryo01,19556,2019.0,"$7,833,333",Houston,$10.833,Batter,False,alvaryo01,135,133,0,0,0,0,0,0,56,0,0,56,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,12775,Brad Miller,32,81,222,-1.2,miller,brad,millebr02,12775,2013.0,"$4,000,000",Texas,FA,Batter,False,millebr02,81,58,0,0,0,0,27,0,28,0,3,29,23
388,19864,Riley Adams,26,48,142,-1.2,adams,riley,adamsri03,19864,2021.0,"$728,800",Washington,NaN,Batter,False,adamsri03,48,43,0,44,1,0,0,0,0,0,0,0,2
389,23378,Jose Barrero,24,48,165,-1.2,barrero,josé,garcijo02,23378,2020.0,"$730,000",Cincinnati,NaN,Batter,False,garcijo02,48,46,0,0,0,0,0,48,0,0,0,0,0
390,11680,Elias Diaz,31,105,351,-1.5,díaz,elías,diazel01,11680,2015.0,"$5,500,000",Colorado,$6.000,Batter,False,diazel01,105,94,0,104,0,0,0,0,0,0,0,0,0


In [97]:
pitcher_data

,IDfg,Name,Age,G,IP,WAR,name_last,name_first,key_bbref,key_fangraphs,mlb_played_first,2023,shortName,2024,SimplePosition,S2
0,16149,Aaron Nola,29,32,205.0,6.3,nola,aaron,nolaaa01,16149,2015.0,"$16,250,000",Philadelphia,FA,Starter,False
1,16137,Carlos Rodon,29,31,178.0,6.2,rodón,carlos,rodonca01,16137,2015.0,"$22,833,333",NY Yankees,$27.833,Starter,False
2,8700,Justin Verlander,39,28,175.0,6.0,verlander,justin,verlaju01,8700,2005.0,"$43,333,333",NY Mets,$43.333,Starter,False
3,18684,Sandy Alcantara,26,32,228.2,5.8,alcántara,sandy,alcansa01,18684,2017.0,"$6,300,000",Miami,$9.300,Starter,False
4,14107,Kevin Gausman,31,31,174.2,5.7,gausman,kevin,gausmke01,14107,2013.0,"$21,000,000",Toronto,$24.000,Starter,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,18694,Kolby Allard,24,10,21.0,-0.7,allard,kolby,allarko01,18694,2018.0,"$750,000",Atlanta,A1,Reliever,False
449,17490,Genesis Cabrera,25,39,44.2,-0.8,cabrera,génesis,cabrege01,17490,2019.0,"$950,000",St. Louis,A2,Reliever,False
450,16933,Elieser Hernandez,27,20,62.1,-0.9,hernández,elieser,hernael01,16933,2018.0,"$1,600,000",NY Mets,A3,Reliever,False
451,6986,Ian Kennedy,37,57,50.1,-1.0,kennedy,ian,kenneia01,6986,2007.0,"$2,250,000",Texas,FA,Reliever,False


In [107]:
pitcher_data['2023'] = pitcher_data['2023'].replace('[\$,]', '', regex=True).astype(float)
batter_data['2023'] = batter_data['2023'].replace('[\$,]', '', regex=True).astype(float)

In [126]:
def calculate_2024Est(row):
    multipliers = {
        ('Batter', False): {'A1': 0, 'A2': 1.36, 'A3': 2.13, 'A4': 3.59},
        ('Batter', True): {'A1': 1.08, 'A2': 1.86, 'A3': 2.66, 'A4': 4.19},
        ('Starter', False): {'A1': 0, 'A2': 1.38, 'A3': 2.35, 'A4': 3.34},
        ('Starter', True): {'A1': 1.11, 'A2': 1.97, 'A3': 2.97, 'A4': 3.88},
        ('Reliever', False): {'A1': 0, 'A2': 1.79, 'A3': 3.98, 'A4': 5.61},
        ('Reliever', True): {'A1': 1.57, 'A2': 3.11, 'A3': 3.98, 'A4': 7.60}
    }
    position = row['SimplePosition']
    s2 = row['S2']
    multiplier_key = (position, s2)
    if pd.isna(row['2024']):
        return 0
    elif '$' in row['2024']:
        return float(row['2024'].replace('$', '').replace(',', '')) * 1000
         
    else: 
        multiplier = multipliers.get(multiplier_key, {}).get(row['2024'], 0)  # Default to 0 if not found 
        if row['WAR'] >= 0:
            return 720000 + row['WAR'] * multiplier * 1000000
        else:
            return 720000

# Apply the function to create the '2024Est' column
pitcher_data['2024Est'] = batter_data.apply(calculate_2024Est, axis=1)
batter_data['2024Est'] = batter_data.apply(calculate_2024Est, axis=1)


In [127]:
df[df['S2'] == True]

,IDfg,Name,Age,G,IP,WAR,name_last,name_first,key_bbref,key_fangraphs,mlb_played_first,2023,shortName,2024,SimplePosition,S2,2024Est
6,13743,Max Fried,28,30,185.1,5.0,fried,max,friedma01,13743,2017.0,13500000.0,Atlanta,A4,Starter,True,20120000.0
10,17295,Framber Valdez,28,31,201.1,4.4,valdez,framber,valdefr01,17295,2018.0,6800000.0,Houston,A3,Starter,True,13788000.0
16,26410,Alek Manoah,24,31,196.2,4.1,manoah,alek,manoaal01,26410,2021.0,745650.0,Toronto,A1,Starter,True,5271000.0
22,19447,Patrick Sandoval,25,27,148.2,3.7,sandoval,patrick,sandopa02,19447,2019.0,2750000.0,LA Angels,A2,Starter,True,8009000.0
23,16162,Brandon Woodruff,29,27,153.1,3.6,woodruff,brandon,woodrbr01,16162,2017.0,10800000.0,Milwaukee,A4,Starter,True,14688000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,15042,Tim Mayza,30,63,48.2,0.1,mayza,tim,mayzati01,15042,2017.0,2100000.0,Toronto,A3,Reliever,True,1118000.0
358,16502,Ryan Yarbrough,30,20,80.0,0.0,yarbrough,ryan,yarbrry01,16502,2018.0,3000000.0,Kansas City,A4,Starter,True,720000.0
391,19742,Anthony Bender,27,22,19.1,-0.1,bender,anthony,bendean01,19742,2021.0,738500.0,Miami,A1,Reliever,True,720000.0
411,22713,Carlos Hernandez,25,27,56.0,-0.1,hernández,carlos,hernaca04,22713,2020.0,730250.0,Kansas City,A1,Reliever,True,720000.0


In [257]:
batter_data.to_csv('../Data/batterDataProduction.csv')
pitcher_data.to_csv('../Data/pitcherDataProduction.csv')

In [245]:
# To work on later 
# From https://blogs.fangraphs.com/an-arbitration-compensation-update/
#Player Type	$/WAR Arb1	$/WAR Arb2	$/WAR Arb3
#Batter	$1.36	$2.13	$3.59
#Starter	$1.38	$2.35	$3.34
#Reliever	$1.79	$3.98	$5.61